In [ ]:
import pandas as pd
vk = pd.read_csv('/Users/olgazdanova/Downloads/processed_vk_posts_only.csv')
vk

In [ ]:
vk_new = vk.copy()

import json

# Функция для извлечения данных из JSON
def extract_json_data(row):
    try:
        # Десериализация JSON
        json_data = json.loads(row['processed_text'])
        
        # Извлечение данных
        setiment = json_data.get('setiment', None)
        symptoms = json_data.get('symptoms', [])
        packaging_damage = json_data.get('packaging_damage', None)
        
        # Извлечение всех reported_symptom через запятую
        reported_symptoms = ", ".join([symptom['reported_symptom'] for symptom in symptoms]) if symptoms else None
        
        return pd.Series([setiment, reported_symptoms, packaging_damage])
    except Exception as e:
        print(f"Ошибка при обработке строки: {e}")
        return pd.Series([None, None, None])

# Применение функции к DataFrame
vk_new[['setiment', 'reported_symptom', 'packaging_damage']] = vk_new.apply(extract_json_data, axis=1)

# Сохранение обновлённого DataFrame в CSV
vk_new.to_csv('vk_new.csv', index=False)

In [ ]:
vk_new_ = vk_new.copy()
vk_new_ = vk_new_.drop(columns='processed_text')
vk_new_.to_csv('vk_new.csv', index=False)

In [ ]:

# Создание пустого списка для хранения данных
data = []

# Проход по каждой строке в DataFrame
for index, row in vk.iterrows():
    # Десериализация JSON строки
    json_data = json.loads(row['processed_text'])
    
    # Извлечение данных из JSON
    symptoms = json_data.get('symptoms', [])
    
    # Проход по каждому симптому
    for symptom in symptoms:
        # Создание новой строки с данными
        new_row = {
            'date': row['post_date'],
            'row_id': row['Unnamed: 0'],
            'reported_symptom': symptom.get('reported_symptom', ''),
            'meddra_term': symptom.get('meddra_term', ''),
            'umls_id': symptom.get('umls_id', ''),
            'concept_type': symptom.get('concept_type', ''),
            'stitch_ids': symptom.get('stitch_ids', '')
        }
        
        # Добавление новой строки в список
        data.append(new_row)

# Создание нового DataFrame из списка данных
new_df_vk = pd.DataFrame(data)

# Сохранение нового DataFrame в CSV файл
new_df_vk.to_csv('new_vk_file.csv', index=False)